<a href="https://colab.research.google.com/github/gaojundong/NMA-Transfer_Learning/blob/main/Tansfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git config --global user.name "gaojundong"
!git config --global user.email "d412@qq.com"


In [2]:
import os
from getpass import getpass
user = getpass('GitHub user')
password = getpass('GitHub password')
os.environ['GITHUB_AUTH'] = user + ':' + password


GitHub user··········
GitHub password··········


GitHub user··········
GitHub password··········
Initialized empty Git repository in /content/.git/
[master (root-commit) beeb93d] first commit
 19 files changed, 50710 insertions(+)
 create mode 100644 .config/.last_opt_in_prompt.yaml
 create mode 100644 .config/.last_survey_prompt.yaml
 create mode 100644 .config/.last_update_check.json
 create mode 100644 .config/active_config
 create mode 100644 .config/config_sentinel
 create mode 100644 .config/configurations/config_default
 create mode 100644 .config/gce
 create mode 100644 .config/logs/2022.07.06/13.20.41.536125.log
 create mode 100644 .config/logs/2022.07.06/13.21.04.313145.log
 create mode 100644 .config/logs/2022.07.06/13.21.24.463860.log
 create mode 100644 .config/logs/2022.07.06/13.21.33.528230.log
 create mode 100644 .config/logs/2022.07.06/13.21.53.919535.log
 create mode 100644 .config/logs/2022.07.06/13.21.54.713715.log
 create mode 100755 sample_data/README.md
 create mode 100755 sample_data/anscombe.json
 create mode